![Neptune + Keras](https://neptune.ai/wp-content/uploads/2023/09/keras.svg)

# Neptune + Keras

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/keras/notebooks/Neptune_Keras.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/keras/notebooks/Neptune_Keras.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/showcase/org/tf-keras-integration/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=Overview-9b8fb3c0-be33-4c15-bef2-2bcdd2c642eb&shortId=TFK-9&type=run">
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/keras/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

This guide will show you how to:

* Initialize Neptune and create a `run`,
* Create a `NeptuneCallback()`,
* Log model training metrics to Neptune using `NeptuneCallback()`
* Log additional metadata like hyperaparameters, images, model diagram and weights to Neptune.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U -q neptune[tensorflow-keras] tensorflow

## Import libraries

In [ ]:
import tensorflow as tf

## Define your model, data loaders, and optimizer

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation=tf.keras.activations.relu),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation=tf.keras.activations.softmax),
    ]
)

optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.005,
    momentum=0.4,
)

model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

## Quickstart

### Start a run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in the public project [common/tf-keras-integration](https://app.neptune.ai/o/common/org/tf-keras-integration). **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    tags=["notebook", "simple"],  # optional
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. To copy the project path, open the settings menu and select **Details & privacy**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
import neptune

run = neptune.init_run(
    project="common/tf-keras-integration",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    tags=["notebook", "simple"],
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

### Add NeptuneCallback() to model.fit()

In [ ]:
from neptune.integrations.tensorflow_keras import NeptuneCallback

neptune_cbk = NeptuneCallback(
    run=run,
    log_on_batch=True,
    log_model_diagram=False,  # Requires pydot to be installed
)

model.fit(x_train, y_train, epochs=5, batch_size=64, callbacks=[neptune_cbk])

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

### Explore results in the Neptune app

Go to the run link and explore charts that were created from the metric values you logged. 

## More Options

You can log way more than just metrics to Neptune. 

Let's go over some of those options here. 

Initialize Neptune run, callback, and set up model

In [ ]:
run = neptune.init_run(
    project="common/tf-keras-integration",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    tags=["notebook", "more options"],
)

neptune_cbk = NeptuneCallback(
    run=run,
    log_on_batch=True,
    log_model_diagram=False,  # Requires pydot to be installed
)

params = {"lr": 0.01, "momentum": 0.8, "epochs": 5, "batch_size": 32}

optimizer = tf.keras.optimizers.SGD(learning_rate=params["lr"], momentum=params["momentum"])

model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

### Log hyperparameters

If you want to log hyperparameters of your model training run you just need to pass them to the `base_namespace` of your choice.

In [ ]:
run["hyper-parameters"] = params

### Upload model

#### Upload checkpoints at the end of each epoch

Initialize Keras' [`ModelCheckpoint`](https://keras.io/api/callbacks/model_checkpoint/) callback.

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_cbk = ModelCheckpoint(
    "checkpoints/ep{epoch:02d}-acc{accuracy:.3f}.keras",
    save_best_only=False,
    save_weights_only=False,
    save_freq="epoch",
)

Fit model using both Neptune and Checkpoint callbacks

In [ ]:
model.fit(
    x_train,
    y_train,
    epochs=params["epochs"],
    batch_size=params["batch_size"],
    callbacks=[neptune_cbk, checkpoint_cbk],
)

Upload checkpoints to Neptune

In [ ]:
run["checkpoints"].upload_files("checkpoints")

#### Upload final model file

In [ ]:
model.save("my_model.keras")

run["saved_model"].upload("my_model.keras")

### Log test sample images

In [ ]:
from neptune.types import File

for image, label in zip(x_test[:10], y_test[:10]):
    prediction = model.predict(image[None], verbose=0)
    predicted = prediction.argmax()
    desc = f"label : {label} | predicted : {predicted}"
    run["visualization/test_prediction"].append(File.as_image(image), description=desc)

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

## Explore results in the Neptune app

You can also check out an [example run](https://app.neptune.ai/o/showcase/org/tf-keras-integration/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=Overview-9b8fb3c0-be33-4c15-bef2-2bcdd2c642eb&shortId=TFK-9&type=run).